# Boosting com o CoutoBoost

In [1]:
import pandas as pd

In [2]:
df_original = pd.read_csv("tic-tac-toe.data", names=[0, 1, 2, 3, 4, 5, 6, 7, 8, "Result"])
df_original.head()

,0,1,2,3,4,5,6,7,8,Result
0,x,x,x,x,o,o,x,o,o,positive
1,x,x,x,x,o,o,o,x,o,positive
2,x,x,x,x,o,o,o,o,x,positive
3,x,x,x,x,o,o,o,b,b,positive
4,x,x,x,x,o,o,b,o,b,positive


In [3]:
print("Exemplos x características: ", (df_original.shape[0], df_original.shape[1]-1))
positive_ratio = len(df_original.loc[df_original["Result"] == "positive"]) / df_original.shape[0]
negative_ratio = 1 - positive_ratio
print(f"Taxa de positivos: {positive_ratio:.2f}")
print(f"Taxa de negativos: {negative_ratio:.2f}")

Exemplos x características:  (958, 9)
Taxa de positivos: 0.65
Taxa de negativos: 0.35


## Preparação dos dados

O algoritmo `CoutoBoostClassifier` implementado é uma versão simplificada do AdaBoost que espera receber as entradas com características binárias e a saída $\epsilon \{-1, +1\}$.



In [4]:
from sklearn.model_selection import train_test_split


# 1. muda a ordem das colunas colocando o label primeiro (para próximo passo)
cols = df_original.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df_original[cols]


# 2. valor numérico para rótulo
df["Result"] = df["Result"].map({
    "positive":  1,
    "negative": -1
})

# 3. transforma cada coluna de feature em 3 colunas binárias
df = pd.get_dummies(df, columns=[0, 1, 2, 3, 4, 5, 6, 7, 8])

# 4. embaralha
df = df.sample(frac=1)

# 5. DataFrame >> ndarray
# 6. X, y
dados = df.to_numpy()
y = dados[:, 0]
X = dados[:, 1:]

# 7. particiona em dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.2)



Similar aos algoritmos do pacote `sklearn`, o `CoutoBoostClassifier` tem em sua interface pública os métodos `fit(X, y)` e `predict(X)`.

In [5]:
from couto_boost import CoutoBoostClassifier
from sklearn.metrics import accuracy_score

clf = CoutoBoostClassifier(max_estimators=2000)
clf.fit(X_train, y_train)



all_stumps:  108
samples =  766
Peso inicial =  0.0013054830287206266
Iteração t= 0
Iteração t= 1
Iteração t= 2
Iteração t= 3
Iteração t= 4
Iteração t= 5
Iteração t= 6
Iteração t= 7
Iteração t= 8
Iteração t= 9
Iteração t= 10
Iteração t= 11
Iteração t= 12
Iteração t= 13
Iteração t= 14
Iteração t= 15
Iteração t= 16
Iteração t= 17
Iteração t= 18
Iteração t= 19
Iteração t= 20
Iteração t= 21
Iteração t= 22
Iteração t= 23
Iteração t= 24
Iteração t= 25
Iteração t= 26
Iteração t= 27
Iteração t= 28
Iteração t= 29
Iteração t= 30
Iteração t= 31
Iteração t= 32
Iteração t= 33
Iteração t= 34
Iteração t= 35
Iteração t= 36
Iteração t= 37
Iteração t= 38
Iteração t= 39
Iteração t= 40
Iteração t= 41
Iteração t= 42
Iteração t= 43
Iteração t= 44
Iteração t= 45
Iteração t= 46
Iteração t= 47
Iteração t= 48
Iteração t= 49
Iteração t= 50
Iteração t= 51
Iteração t= 52
Iteração t= 53
Iteração t= 54
Iteração t= 55
Iteração t= 56
Iteração t= 57
Iteração t= 58
Iteração t= 59
Iteração t= 60
Iteração t= 61
Iteração t

Iteração t= 517
Iteração t= 518
Iteração t= 519
Iteração t= 520
Iteração t= 521
Iteração t= 522
Iteração t= 523
Iteração t= 524
Iteração t= 525
Iteração t= 526
Iteração t= 527
Iteração t= 528
Iteração t= 529
Iteração t= 530
Iteração t= 531
Iteração t= 532
Iteração t= 533
Iteração t= 534
Iteração t= 535
Iteração t= 536
Iteração t= 537
Iteração t= 538
Iteração t= 539
Iteração t= 540
Iteração t= 541
Iteração t= 542
Iteração t= 543
Iteração t= 544
Iteração t= 545
Iteração t= 546
Iteração t= 547
Iteração t= 548
Iteração t= 549
Iteração t= 550
Iteração t= 551
Iteração t= 552
Iteração t= 553
Iteração t= 554
Iteração t= 555
Iteração t= 556
Iteração t= 557
Iteração t= 558
Iteração t= 559
Iteração t= 560
Iteração t= 561
Iteração t= 562
Iteração t= 563
Iteração t= 564
Iteração t= 565
Iteração t= 566
Iteração t= 567
Iteração t= 568
Iteração t= 569
Iteração t= 570
Iteração t= 571
Iteração t= 572
Iteração t= 573
Iteração t= 574
Iteração t= 575
Iteração t= 576
Iteração t= 577
Iteração t= 578
Iteração

Iteração t= 1030
Iteração t= 1031
Iteração t= 1032
Iteração t= 1033
Iteração t= 1034
Iteração t= 1035
Iteração t= 1036
Iteração t= 1037
Iteração t= 1038
Iteração t= 1039
Iteração t= 1040
Iteração t= 1041
Iteração t= 1042
Iteração t= 1043
Iteração t= 1044
Iteração t= 1045
Iteração t= 1046
Iteração t= 1047
Iteração t= 1048
Iteração t= 1049
Iteração t= 1050
Iteração t= 1051
Iteração t= 1052
Iteração t= 1053
Iteração t= 1054
Iteração t= 1055
Iteração t= 1056
Iteração t= 1057
Iteração t= 1058
Iteração t= 1059
Iteração t= 1060
Iteração t= 1061
Iteração t= 1062
Iteração t= 1063
Iteração t= 1064
Iteração t= 1065
Iteração t= 1066
Iteração t= 1067
Iteração t= 1068
Iteração t= 1069
Iteração t= 1070
Iteração t= 1071
Iteração t= 1072
Iteração t= 1073
Iteração t= 1074
Iteração t= 1075
Iteração t= 1076
Iteração t= 1077
Iteração t= 1078
Iteração t= 1079
Iteração t= 1080
Iteração t= 1081
Iteração t= 1082
Iteração t= 1083
Iteração t= 1084
Iteração t= 1085
Iteração t= 1086
Iteração t= 1087
Iteração t= 10

Iteração t= 1513
Iteração t= 1514
Iteração t= 1515
Iteração t= 1516
Iteração t= 1517
Iteração t= 1518
Iteração t= 1519
Iteração t= 1520
Iteração t= 1521
Iteração t= 1522
Iteração t= 1523
Iteração t= 1524
Iteração t= 1525
Iteração t= 1526
Iteração t= 1527
Iteração t= 1528
Iteração t= 1529
Iteração t= 1530
Iteração t= 1531
Iteração t= 1532
Iteração t= 1533
Iteração t= 1534
Iteração t= 1535
Iteração t= 1536
Iteração t= 1537
Iteração t= 1538
Iteração t= 1539
Iteração t= 1540
Iteração t= 1541
Iteração t= 1542
Iteração t= 1543
Iteração t= 1544
Iteração t= 1545
Iteração t= 1546
Iteração t= 1547
Iteração t= 1548
Iteração t= 1549
Iteração t= 1550
Iteração t= 1551
Iteração t= 1552
Iteração t= 1553
Iteração t= 1554
Iteração t= 1555
Iteração t= 1556
Iteração t= 1557
Iteração t= 1558
Iteração t= 1559
Iteração t= 1560
Iteração t= 1561
Iteração t= 1562
Iteração t= 1563
Iteração t= 1564
Iteração t= 1565
Iteração t= 1566
Iteração t= 1567
Iteração t= 1568
Iteração t= 1569
Iteração t= 1570
Iteração t= 15

Iteração t= 1995
Iteração t= 1996
Iteração t= 1997
Iteração t= 1998
Iteração t= 1999


In [6]:
y_hat = clf.predict(X_test)

score = accuracy_score(y_test, y_hat)
print(score)

0.96875


In [7]:
from sklearn.ensemble import AdaBoostClassifier

clf_baseline = AdaBoostClassifier(n_estimators=2000)
clf_baseline.fit(X_train, y_train)


AdaBoostClassifier(n_estimators=2000)

In [8]:
y_hat_baseline = clf_baseline.predict(X_test)

score_baseline = accuracy_score(y_test, y_hat_baseline)
print(score_baseline)

0.96875
